# Exercise 1: Compute pi on the GPU

## Tasks

1. Write a CPU-program that computes pi using geometry and statistics
2. Port this program to the GPU
3. Optimize

## Solution

<-- see `compute_pi_solution.ipynb`

In [17]:
import numpy as np
import time

rng = np.random.default_rng()

def compute_pi_cpu(n_points):
    x_rand = rng.random(n_points)
    y_rand = rng.random(n_points)

    inside = np.sqrt(x_rand**2 + y_rand**2) <= 1.0
    n_inside = np.sum(inside)

    #n_inside = 0
    #for i in range(n_points):
    #    n_inside += np.sqrt(x_rand[i]**2 + y_rand[i]**2) <= 1.0

    pi = 4*n_inside/n_points

    return pi

In [18]:
%%time

tic = time.time()
print(compute_pi_cpu(100000))
toc = time.time()

print("Time for CPU-version: {:f} seconds".format(toc-tic))

3.13044
Time for CPU-version: 0.003916 seconds
CPU times: total: 0 ns
Wall time: 4.92 ms
